In [1]:

import gc
import os
import torch
import random   

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

from copy import copy
from tqdm import tqdm
from torch.optim import Adam, SGD
from sklearn.metrics import log_loss,mean_squared_error


### Load Data

In [2]:
## https://github.com/JeffSackmann/tennis_wta

DATA_PATH = 'D://Medium'
os.listdir(DATA_PATH)

def load_data():
    return pd.read_csv(os.path.join(DATA_PATH, 'ncaam_sample_data.csv'))

data = load_data()
data.head()


,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_or,opp_dr,opp_ast,opp_to,opp_stl,opp_blk,opp_pf,team_name,opp_name,date
0,2003,68,62,0,0,27,58,3,14,11,...,10,22,8,18,9,2,20,Alabama,Oklahoma,2002-11-14
1,2003,70,63,0,0,26,62,8,20,10,...,20,25,7,12,8,6,16,Memphis,Syracuse,2002-11-14
2,2003,62,68,0,0,22,53,2,10,16,...,14,24,13,23,7,1,22,Oklahoma,Alabama,2002-11-14
3,2003,63,70,0,0,24,67,6,24,9,...,15,28,16,13,4,4,18,Syracuse,Memphis,2002-11-14
4,2003,55,81,-1,0,20,46,3,11,12,...,12,24,12,9,9,3,18,E Washington,Wisconsin,2002-11-15


In [3]:

def feature_eng(df):
    ## need match id. And going to only do one row per game, shuffled between perspective of winner and loser
    df['team_A'] = df[['team_name','opp_name']].copy().apply(lambda x: sorted([x.team_name, x.opp_name])[0],axis=1)
    df['match_id'] = df['date'].copy().astype(str)+'_'+df['team_A'].copy()
    df = df.sample(frac=1)
    df = df.drop_duplicates(subset=['match_id'], keep='first')
    df = df.drop('team_A',axis=1)
    df = df.sort_values(by='date').reset_index(drop=True)
    
    targets = ['score_diff_pct','total_reb_pct','score_total']
    stats = ['score_diff_pct','oreb_pct','dreb_pct','fgm3_diff_pct','ftm_diff_pct','score_total','blk_pct_diff','ast_pct_diff','to_pct_diff', 'agg_pct_diff']
    ## just mirror stats
    opp_stats = ['opp_'+stat for stat in stats]
    meta = ['is_home','days_rest', 'opp_days_rest']
    
    ## For NN, good for everything to be on same scale
    ## in my first attempt i divided score diff by score total, etc. I think standard scaler might work better
    df['score_diff_pct'] = (df['team_score'].copy()-df['opp_score'].copy())/(df['team_score'].copy()+df['opp_score'].copy())
    df['oreb_pct'] = (df['team_or'].copy()-df['opp_dr'].copy())/(df['team_or'].copy()+df['opp_dr'].copy())
    df['dreb_pct'] = (df['team_dr'].copy()-df['opp_or'].copy())/(df['team_dr'].copy()+df['opp_or'].copy())
    df['total_reb_pct'] = ((df['team_or'].copy()+df['team_dr'].copy())-(df['opp_dr'].copy()+df['opp_or'].copy()))/((df['team_or'].copy()+df['team_dr'].copy())+(df['opp_dr'].copy()+df['opp_or'].copy()))
    df['fgm3_diff_pct'] = (df['team_fgm3'].copy()-df['opp_fgm3'].copy())/(df['team_fgm3'].copy()+df['opp_fgm3'].copy())
    df['ftm_diff_pct'] = (df['team_ftm'].copy()-df['opp_ftm'].copy())/(df['team_ftm'].copy()+df['opp_ftm'].copy())
    df['score_total'] = (df['team_score'].copy()+df['opp_score'].copy())/265
    df['blk_pct_diff'] = (df['team_blk'].copy()-df['opp_blk'].copy())/(df['team_blk'].copy()+df['opp_blk'].copy())
    df['ast_pct_diff'] = (df['team_ast'].copy()-df['opp_ast'].copy())/(df['team_ast'].copy()+df['opp_ast'].copy())
    df['to_pct_diff'] = (df['team_to'].copy()-df['opp_to'].copy())/(df['team_to'].copy()+df['opp_to'].copy())
    df['agg_pct_diff'] = ((df['team_stl'].copy()+df['team_pf'].copy())-(df['opp_stl'].copy()+df['opp_pf'].copy()))/((df['team_stl'].copy()+df['team_pf'].copy())+(df['opp_stl'].copy()+df['opp_pf'].copy()))
    
    df['opp_score_diff_pct'] = -1*(df['team_score'].copy()-df['opp_score'].copy())/(df['team_score'].copy()+df['opp_score'].copy())
    ## or and d reb swap 
    df['opp_dreb_pct'] = -1*(df['team_or'].copy()-df['opp_dr'].copy())/(df['team_or'].copy()+df['opp_dr'].copy())
    df['opp_oreb_pct'] = -1*(df['team_dr'].copy()-df['opp_or'].copy())/(df['team_dr'].copy()+df['opp_or'].copy())
    ##
    df['opp_total_reb_pct'] = -1*((df['team_or'].copy()+df['team_dr'].copy())-(df['opp_dr'].copy()+df['opp_or'].copy()))/((df['team_or'].copy()+df['team_dr'].copy())+(df['opp_dr'].copy()+df['opp_or'].copy()))
    df['opp_fgm3_diff_pct'] = -1*(df['team_fgm3'].copy()-df['opp_fgm3'].copy())/(df['team_fgm3'].copy()+df['opp_fgm3'].copy())
    df['opp_ftm_diff_pct'] = -1*(df['team_ftm'].copy()-df['opp_ftm'].copy())/(df['team_ftm'].copy()+df['opp_ftm'].copy())
    df['opp_score_total'] = -1*(df['team_score'].copy()+df['opp_score'].copy())/265
    df['opp_blk_pct_diff'] = -1*(df['team_blk'].copy()-df['opp_blk'].copy())/(df['team_blk'].copy()+df['opp_blk'].copy())
    df['opp_ast_pct_diff'] = -1*(df['team_ast'].copy()-df['opp_ast'].copy())/(df['team_ast'].copy()+df['opp_ast'].copy())
    df['opp_to_pct_diff'] = -1*(df['team_to'].copy()-df['opp_to'].copy())/(df['team_to'].copy()+df['opp_to'].copy())
    df['opp_agg_pct_diff'] = -1*((df['team_stl'].copy()+df['team_pf'].copy())-(df['opp_stl'].copy()+df['opp_pf'].copy()))/((df['team_stl'].copy()+df['team_pf'].copy())+(df['opp_stl'].copy()+df['opp_pf'].copy()))
    
    df['date'] = pd.to_datetime(df['date'].copy())
    df['last_played'] = df.groupby(['team_name'])['date'].transform('shift')
    df['last_played'] = df['last_played'].fillna(df['date'].copy()-pd.Timedelta(days=42))
    df['opp_last_played'] = df.groupby(['opp_name'])['date'].transform('shift')
    df['opp_last_played'] = df['opp_last_played'].fillna(df['date'].copy()-pd.Timedelta(days=42))
    df['days_rest'] = ((df['date'].copy()-df['last_played'].copy()).dt.days)/365
    df['opp_days_rest'] = ((df['date'].copy()-df['opp_last_played'].copy()).dt.days)/365
    df = df.drop(columns=['last_played','opp_last_played'])
    
    ## only a handful, mostly blocks 
    df[stats] = df[stats].fillna(0.5)
    df[opp_stats] = df[opp_stats].fillna(0.5)
    
    df['result'] = np.where(df['team_score'].copy()>df['opp_score'].copy(), 1, 0)
    
    return df, targets, stats, opp_stats, meta

data, targets, stats, opp_stats, meta = feature_eng(data)


In [4]:

teams = set(data['team_name'].unique()).union(set(data['opp_name'].unique()))


In [5]:

print(list(data))
data.head()


['season', 'team_score', 'opp_score', 'is_home', 'numot', 'team_fgm', 'team_fga', 'team_fgm3', 'team_fga3', 'team_ftm', 'team_fta', 'team_or', 'team_dr', 'team_ast', 'team_to', 'team_stl', 'team_blk', 'team_pf', 'opp_fgm', 'opp_fga', 'opp_fgm3', 'opp_fga3', 'opp_ftm', 'opp_fta', 'opp_or', 'opp_dr', 'opp_ast', 'opp_to', 'opp_stl', 'opp_blk', 'opp_pf', 'team_name', 'opp_name', 'date', 'match_id', 'score_diff_pct', 'oreb_pct', 'dreb_pct', 'total_reb_pct', 'fgm3_diff_pct', 'ftm_diff_pct', 'score_total', 'blk_pct_diff', 'ast_pct_diff', 'to_pct_diff', 'agg_pct_diff', 'opp_score_diff_pct', 'opp_dreb_pct', 'opp_oreb_pct', 'opp_total_reb_pct', 'opp_fgm3_diff_pct', 'opp_ftm_diff_pct', 'opp_score_total', 'opp_blk_pct_diff', 'opp_ast_pct_diff', 'opp_to_pct_diff', 'opp_agg_pct_diff', 'days_rest', 'opp_days_rest', 'result']


,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_fgm3_diff_pct,opp_ftm_diff_pct,opp_score_total,opp_blk_pct_diff,opp_ast_pct_diff,opp_to_pct_diff,opp_agg_pct_diff,days_rest,opp_days_rest,result
0,2003,63,70,0,0,24,67,6,24,9,...,0.142857,0.052632,-0.501887,-0.200000,0.391304,0.040000,-0.043478,0.115068,0.115068,0
1,2003,62,68,0,0,22,53,2,10,16,...,0.200000,-0.185185,-0.490566,-0.333333,0.238095,0.121951,0.000000,0.115068,0.115068,0
2,2003,61,73,0,0,22,73,3,26,14,...,0.454545,0.096774,-0.505660,-0.428571,0.250000,-0.090909,0.090909,0.115068,0.115068,0
3,2003,50,56,0,0,18,49,6,22,8,...,-0.333333,0.360000,-0.400000,-0.200000,0.100000,-0.225806,0.084746,0.115068,0.115068,0
4,2003,77,71,0,0,30,61,6,14,11,...,0.000000,0.214286,-0.558491,-0.600000,0.000000,-0.166667,-0.066667,0.115068,0.115068,1


### Run Classic Elo

In [6]:

class StatefulSystem:
    def __init__(self):
        self.history = []  # to store history of predictions and results

    def predict_1v1(self, player1, player2, **kwargs):
        raise NotImplementedError  # This method should be implemented in child classes

    def update_1v1(self, player1, player2, result, **kwargs):
        raise NotImplementedError  # This method should be implemented in child classes
        
class PlayerNode():
    def __init__(self, rating):
        self.rating=rating
        
class EloNode(PlayerNode):
    def __init__(self,_id, name, rating=1500):
        super().__init__(rating)
        self._id = _id
        self.name = name
        self.rating = rating
        self.rank = 200
        
class EloSystem(StatefulSystem):
    def __init__(self, k_factor, meta_functions=None):
        super().__init__()
        self.history = []
        self.k_factor = k_factor
        ## for edge info like home, days off
        self.meta_functions = meta_functions

    def predict_1v1(self, player1, player2, **kwargs):
        # Meta information can be accessed as dictionary items, e.g., kwargs['is_home'], kwargs['days_off']
        rd = player1.rating - player2.rating
        if self.meta_functions is not None:
            ## add all adjustments for meta information
            for meta_key, meta_function in self.meta_functions.items():
                rd += meta_function(kwargs[meta_key])
        prediction = 1/(1+10**(-rd/400))
        return prediction
    def update_1v1(self, prediction, result):
        points_exchanged = self.k_factor*(result-prediction)
        return points_exchanged
    
    def play_match(self, p1, p2, result, update_both=False, **kwargs):
        prediction = self.predict_1v1(p1, p2, **kwargs)
        ratings_delta = self.update_1v1(prediction, result)
        self.history.append([p1._id, p1.name, p2._id, p2.name, p1.rating, p2.rating, prediction, result, ratings_delta])
        p1.rating+=ratings_delta
        if update_both:
            p2.rating-=ratings_delta
        return p1, p2
    
    def get_history(self):
        return pd.DataFrame(self.history, columns=['p1_id','p1_name','p2_id','p2_name','p1_rating','p2_rating','prediction','result','ratings_delta'])

elo_sys = EloSystem(k_factor=40)
# player_ratings = {_id:EloNode(_id, name, 1500) for _id, name in player_names.items()}
team_ratings = {name:EloNode(i, name, 1500) for i, name in enumerate(teams)}
for index, row in tqdm(data.iterrows(), total=len(data)):
    
    t1 = row['team_name']
    t2 = row['opp_name']
    
    t1_node = team_ratings[t1]
    t2_node = team_ratings[t2]
    
    result = 1 if (row['team_score']-row['opp_score'])>0 else 0

    t1_node, t2_node = elo_sys.play_match(t1_node, t2_node, result, update_both=True)
    
    team_ratings[t1] = t1_node
    team_ratings[t2] = t2_node
    
    
hist = elo_sys.get_history()
hist
    

100%|███████████████████████████████████████████████████████████████████████| 103280/103280 [00:02<00:00, 42719.22it/s]


,p1_id,p1_name,p2_id,p2_name,p1_rating,p2_rating,prediction,result,ratings_delta
0,212,Syracuse,132,Memphis,1500.000000,1500.000000,0.500000,0,-20.000000
1,73,Oklahoma,281,Alabama,1500.000000,1500.000000,0.500000,0,-20.000000
2,118,Villanova,221,Marquette,1500.000000,1500.000000,0.500000,0,-20.000000
3,346,Winthrop,359,N Illinois,1500.000000,1500.000000,0.500000,0,-20.000000
4,133,Texas,251,Georgia,1500.000000,1500.000000,0.500000,1,20.000000
...,...,...,...,...,...,...,...,...,...
103275,328,St Peter's,30,North Carolina,1620.746851,2047.468070,0.078971,0,-3.158858
103276,6,Kansas,95,Miami FL,2155.153018,1959.018606,0.755662,1,9.773529
103277,6,Kansas,118,Villanova,2164.926547,2157.342296,0.510913,1,19.563485
103278,30,North Carolina,353,Duke,2050.626928,2096.882959,0.433823,1,22.647090


In [7]:
from sklearn.metrics import log_loss
## 20: 0.5717748620218386
## 30: 0.5688727434688168
## 40: 0.5684751964171788
## 45: 0.5694965390004977
## 60: 0.5735
grade_cutoff = int(0.2*len(hist))
log_loss(hist[-grade_cutoff:]['result'].values, hist[-grade_cutoff:]['prediction'].values)

0.5686497097006444

In [8]:

rtg_df = pd.DataFrame([[k,v.name, v.rating] for k,v in team_ratings.items()], columns=['id','name','rating'])
rtg_df.sort_values(by='rating', ascending=False).head(20)


,id,name,rating
6,Kansas,Kansas,2198.297987
262,Gonzaga,Gonzaga,2182.581492
118,Villanova,Villanova,2137.778811
186,Baylor,Baylor,2127.109214
216,Houston,Houston,2082.292836
353,Duke,Duke,2074.235869
171,Arizona,Arizona,2061.981370
30,North Carolina,North Carolina,2059.466062
270,Tennessee,Tennessee,2042.358774
274,Arkansas,Arkansas,2038.398964


### Create Pytorch Elo Classes


In [9]:

# One neuron to learn scaling and a sigmoid activation function.
class EloPredictNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1, bias=False)  # No bias so that ratings delta of 0 = 50% win prob

    def forward(self, rd):
        return torch.sigmoid(self.linear(rd))
    
class EloUpdateNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1, bias=False)  # No bias so that matches are symmetrical (no advantage to being player A)
        
    def forward(self, pred_error):
        return self.linear(pred_error)
    
class NetworkNode(PlayerNode):
    def __init__(self,_id, name, rating=0):
        super().__init__(rating)
        self._id = _id
        self.name = name
        self.rating = rating
        self.last_rating = rating
        self.last_error = 0.5



### Let's try learning the weights

In [10]:

## split into train/test
percent_30 = int(0.3*len(data))
train = data.copy()[:-percent_30]
test = data.copy()[-percent_30:]


In [11]:

predict_model = EloPredictNN()
predict_optimizer = Adam(predict_model.parameters(), lr=1e-6)

update_model = EloUpdateNN()
update_optimizer = Adam(update_model.parameters(), lr=1e-6)

batch_size = 24 
num_batches = len(train)//batch_size
num_epochs = 2
best_val_loss = np.inf



In [12]:

## temporal component important so no shuffle
for j in range(num_epochs):
    network_team_ratings = {name:NetworkNode(i, name, 0) for i, name in enumerate(teams)}
    epoch_loss = []
    predict_model.train()
    update_model.train()
    for i in tqdm(range(num_batches-1), total=num_batches-1):
        predict_optimizer.zero_grad()
        update_optimizer.zero_grad()
        train_data = train[i*batch_size:(i+1)*batch_size].copy()

        results = torch.from_numpy(train_data.result.copy().astype('float32').values).view(-1,1)
        
        t1_nodes = [network_team_ratings[k] for k in train_data.team_name.values]
        t2_nodes = [network_team_ratings[k] for k in train_data.opp_name.values]
        
        predict_X = torch.Tensor([t2_nodes[k].rating-t1_nodes[k].rating for k in range(len(t1_nodes))]).view(-1,1)
        
        ## I found that only updating one model at a time seems to give more stable results
        ## worth investigating further
        update_or_predict = np.random.random()
        
        if update_or_predict>0.5:
            ## PREDICT MODEL (easy part)
            ## have to make it symmetrical (p2_rating-p1_rating and p1_rating-p2_rating)
            predictions = predict_model(predict_X)
            
            predict_loss = nn.BCELoss()(predictions, results)
            predict_loss.backward()

            # Update the model's parameters
            ## for the first few iterations, let ratings stabilize
            if (j>0)&(i < 20):
                continue
            elif (j>1)&(i < 100):
                continue
            else:
                predict_optimizer.step()
            # Zero the gradients since PyTorch accumulates them
            predict_optimizer.zero_grad()
        
        else:
            ## UPDATE MODEL (slightly harder)
            ## use previous game's results to calculate loss of the update model
            predictions = predict_model(predict_X)
            predict_optimizer.zero_grad()
            
            ## directionality is important here, last error is simply result - prediction (is negative if the player loses)
            t1_update_X = torch.Tensor([t1_node.last_error for t1_node in t1_nodes]).view(-1,1)
            t2_update_X = torch.Tensor([t2_node.last_error for t2_node in t2_nodes]).view(-1,1)

            t1_update_predictions = update_model(t1_update_X)
            t2_update_predictions = update_model(t2_update_X)

            ## want both baseline predict model (no update) and predict model with updates. Compare to create loss
            baseline_X = torch.Tensor([t2_nodes[k].last_rating-t1_nodes[k].last_rating for k in range(len(t1_nodes))]).view(-1,1)
            update_model_X = torch.Tensor([t2_nodes[k].last_rating+t2_update_predictions[k] - t1_nodes[k].last_rating+t1_update_predictions[k] for k in range(len(t1_nodes))]).view(-1,1)

            baseline_predictions = predict_model(baseline_X)
            update_model_predictions = predict_model(update_model_X)
            baseline_BCE = nn.BCELoss()(baseline_predictions, results)
            update_model_BCE = nn.BCELoss()(update_model_predictions, results)
            ## lower is better, has to be negative if improving ratings
            update_loss = update_model_BCE-baseline_BCE
            update_loss.backward()
            ## skip the first few iterations of the later epochs to give some time for ratings to develop
            if (j>0)&(i < 25):
                continue
            elif (j>1)&(i < 100):
                continue
            else:
                update_optimizer.step()
            update_optimizer.zero_grad()
        
        ## now just update the nodes with the predictions
        ## in Elo this step is k*(result-predictions)
        ## we are finding k
        pred_error = results - predictions
        epoch_loss.extend(list(pred_error[:,0].detach().numpy()))
        updates = update_model(pred_error)
        new_ratings = [t1_nodes[k].rating+updates[k] for k in range(len(t1_nodes))]

        for l, team_node in enumerate(t1_nodes):
            team_node.last_rating = copy(team_node.rating)
            team_node.last_error = pred_error[l, 0]
            team_node.rating = float(new_ratings[l])
            network_team_ratings[team_node._id] = copy(team_node)
            
    ## calculate val
    predict_model.eval()
    update_model.eval()
    
    network_team_ratings = {name:NetworkNode(_id, name, 0) for _id, name in enumerate(teams)}

    nn_history = []
    for index, row in tqdm(data.iterrows(), total=len(data)):
        
        t1_id, t2_id = row['team_name'], row['opp_name']
        
        t1_nn_node = network_team_ratings[t1_id]
        t2_nn_node = network_team_ratings[t2_id]
        result = row['result']

        rtg_diff = t2_nn_node.rating-t1_nn_node.rating
        nn_prediction = predict_model(torch.Tensor([[rtg_diff]]))
        error = result - nn_prediction
        rtg_update = update_model(torch.Tensor([[error]]))
        nn_history.append([t1_nn_node._id, t1_nn_node.name, t2_nn_node._id, t2_nn_node.name, float(t1_nn_node.rating), float(t2_nn_node.rating), float(nn_prediction), result, float(rtg_update)])
        t1_nn_node.rating =  t1_nn_node.rating + rtg_update
        t2_nn_node.rating =  t2_nn_node.rating - rtg_update
        
        network_team_ratings[t1_id] = t1_nn_node
        network_team_ratings[t2_id] = t2_nn_node

    nn_hist = pd.DataFrame(nn_history, columns=['t1_id','t1_name','t2_id','t2_name','t1_rating','t2_rating','prediction','result','ratings_delta'])
    val_loss = log_loss(nn_hist[-grade_cutoff:]['result'].values, nn_hist[-grade_cutoff:]['prediction'].values)
    print(f"NN validation log loss (hopefully < 0.69 and >0.55): {val_loss}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print("Saving parameters...")
        torch.save(predict_model.state_dict(), 'elo_predict_nn_best_model.pth')
        torch.save(update_model.state_dict(), 'elo_update_nn_best_model.pth')
        
    

100%|███████████████████████████████████████████████████████████████████████| 103280/103280 [00:07<00:00, 13308.51it/s]


NN validation log loss (hopefully < 0.69 and >0.55): 18.186456775669807
Saving parameters...


100%|███████████████████████████████████████████████████████████████████████| 103280/103280 [00:07<00:00, 13178.10it/s]

NN validation log loss (hopefully < 0.69 and >0.55): 18.174469648354506
Saving parameters...


In [13]:
predict_model = EloPredictNN()
predict_model.load_state_dict(torch.load('elo_predict_nn_best_model.pth'))
update_model = EloUpdateNN()
update_model.load_state_dict(torch.load('elo_update_nn_best_model.pth'))


<All keys matched successfully>

In [14]:
rtg_df = pd.DataFrame([[k,v.name, float(v.rating)] for k,v in network_team_ratings.items()], columns=['id','name','rating'])
rtg_df.sort_values(by='rating', ascending=False).head()


,id,name,rating
163,Chicago St,Chicago St,396.918915
74,San Jose St,San Jose St,388.031921
31,MD E Shore,MD E Shore,379.620850
296,Ark Pine Bluff,Ark Pine Bluff,374.206635
278,Fordham,Fordham,347.904633


In [15]:

predict_model.eval()
update_model.eval()


EloUpdateNN(
  (linear): Linear(in_features=1, out_features=1, bias=False)
)

In [16]:

elo_sys = EloSystem(k_factor=50)

player_ratings = {name:EloNode(_id, name, 1500) for _id, name in enumerate(teams)}
network_player_ratings = {name:NetworkNode(_id, name, 0) for _id, name in enumerate(teams)}

nn_history = []
data = data.sort_values(by=['date','team_name']).reset_index(drop=True)
for index, row in tqdm(data.iterrows(), total=len(data)):
    ## randomize who is p1 and who is p2
    p1_id, p2_id = row['team_name'], row['opp_name']
    p1_node = player_ratings[p1_id]
    p2_node = player_ratings[p2_id]
    result = row['result']
    p1_node, p2_node = elo_sys.play_match(p1_node, p2_node, result)
    
    p1_nn_node = network_player_ratings[p1_id]
    p2_nn_node = network_player_ratings[p2_id]
    
    rtg_diff = p2_nn_node.rating-p1_nn_node.rating
    nn_prediction = predict_model(torch.Tensor([[rtg_diff]]))
    error = result - nn_prediction
    rtg_update = update_model(torch.Tensor([[error]]))
    nn_history.append([row['date'], row['match_id'], p1_nn_node._id, p1_nn_node.name, p2_nn_node._id, p2_nn_node.name, float(p1_nn_node.rating), float(p2_nn_node.rating), float(nn_prediction), result, float(rtg_update)])
    p1_nn_node.rating += rtg_update
    p2_nn_node.rating -= rtg_update
    
    player_ratings[p1_id] = p1_node
    player_ratings[p2_id] = p2_node
    
    
hist = elo_sys.get_history()
nn_hist = pd.DataFrame(nn_history, columns=['date','match_id','p1_id','p1_name','p2_id','p2_name','p1_rating','p2_rating','prediction','result','ratings_delta'])



100%|███████████████████████████████████████████████████████████████████████| 103280/103280 [00:08<00:00, 12123.54it/s]


In [17]:


grade_cutoff = int(0.2*len(hist))
print(len(hist),len(nn_hist))
reg_log_loss = log_loss(hist[-grade_cutoff:]['result'].values, hist[-grade_cutoff:]['prediction'].values)
nn_log_loss = log_loss(nn_hist[-grade_cutoff:]['result'].values, nn_hist[-grade_cutoff:]['prediction'].values)
print("reg log loss: ", reg_log_loss)
print("nn log loss: ", nn_log_loss)



103280 103280
reg log loss:  0.5863064816065223
nn log loss:  18.172468894528727



### More Advanced Model



In [18]:

vec_size = 3

class PairwisePredictNN(nn.Module):
    """
    takes [
    player 2 vec - player 1 vec (distance), 
    meta
    ] and predicts match outcome
    """
    def __init__(self, input_size, hidden_dim, output_size):
        super().__init__()
        self.input_size=input_size
        self.hidden_dim=hidden_dim
        self.output_size=output_size
        self.linear = nn.Linear(input_size, self.hidden_dim)
        self.linear2 = nn.Linear(self.hidden_dim, self.output_size)

    def forward(self, x):
        x = F.relu(self.linear(x))
        x = self.linear2(x)
        ## ensure total is non negative
#         x[:, 2] = torch.nn.functional.softplus(x[:, 2])
        return x
    
class PairwiseUpdateNN(nn.Module):
    def __init__(self, input_size, hidden_dim, player_vec_size):
        super().__init__()
        """
        takes [
        player 2 vec - player 1 vec (distance), 
        meta,
        stat_results
        ] and updates player vecs symmetrically
        """
        self.input_size=input_size
        self.hidden_dim=hidden_dim
        self.player_vec_size=player_vec_size
        self.linear = nn.Linear(self.input_size, self.hidden_dim) 
        self.linear2 = nn.Linear(self.hidden_dim, self.player_vec_size)
        
    def forward(self, x):
        x = F.relu(self.linear(x))
        x = self.linear2(x)
        return torch.tanh(x) *0.5
    
class NetworkNode(PlayerNode):
    def __init__(self,_id, name, rating=torch.zeros(vec_size)):
        super().__init__(rating)
        self._id = _id
        self.name = name
        self.rating = rating
        self.last_rating = rating
        ### vector created at end of last match (used to train update model)
        ### distance, meta, 
        self.last_vector = torch.cat([torch.zeros(vec_size), torch.zeros(len(meta)), torch.zeros(len(stats))])
        
    def get_last_update_input(self):
        return self.last_vector
    
    def pregame_state(self):
        pregame_state = [self._id, self.name]
        pregame_state.extend(list(self.last_rating.detach().numpy()))
        return pregame_state
        


print(vec_size+len(meta)+len(stats))
test = NetworkNode(100, "Blake")


16


In [19]:

data.head()


,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_fgm3_diff_pct,opp_ftm_diff_pct,opp_score_total,opp_blk_pct_diff,opp_ast_pct_diff,opp_to_pct_diff,opp_agg_pct_diff,days_rest,opp_days_rest,result
0,2003,62,68,0,0,22,53,2,10,16,...,0.200000,-0.185185,-0.490566,-0.333333,0.238095,0.121951,0.000000,0.115068,0.115068,0
1,2003,63,70,0,0,24,67,6,24,9,...,0.142857,0.052632,-0.501887,-0.200000,0.391304,0.040000,-0.043478,0.115068,0.115068,0
2,2003,77,71,0,0,30,61,6,14,11,...,0.000000,0.214286,-0.558491,-0.600000,0.000000,-0.166667,-0.066667,0.115068,0.115068,1
3,2003,61,73,0,0,22,73,3,26,14,...,0.454545,0.096774,-0.505660,-0.428571,0.250000,-0.090909,0.090909,0.115068,0.115068,0
4,2003,50,56,0,0,18,49,6,22,8,...,-0.333333,0.360000,-0.400000,-0.200000,0.100000,-0.225806,0.084746,0.115068,0.115068,0


In [20]:

predict_model = PairwisePredictNN(
    input_size=vec_size+len(meta),
    hidden_dim=3,
    output_size=3
)
predict_optimizer = Adam(predict_model.parameters(), lr=1e-4)

update_model = PairwiseUpdateNN(
    input_size = vec_size+len(meta)+len(stats),
    hidden_dim = 9,
    player_vec_size = 3
)
update_optimizer = Adam(update_model.parameters(), lr=1e-4)



In [21]:

train = data.copy()[:-percent_30]
test = data.copy()[-percent_30:]
batch_size = 32
num_batches = len(train)//batch_size
num_epochs = 7
best_val_loss = np.inf



In [22]:


for j in range(num_epochs):
    network_team_ratings = {name:NetworkNode(_id, name, (torch.rand(vec_size)-0.5)*0.05) for _id, name in enumerate(teams)}
    epoch_loss = []
    predict_model.train()
    update_model.train()
    for i in tqdm(range(num_batches-1), total=num_batches-1):
        predict_optimizer.zero_grad()
        update_optimizer.zero_grad()
        train_data = train[i*batch_size:(i+1)*batch_size].copy()
        
        t1_nodes= [network_team_ratings[team] for team in train_data.team_name.values]
        t2_nodes= [network_team_ratings[team] for team in train_data.opp_name.values]
        
        distances_1 = torch.stack([(t2_nodes[i].rating-t1_nodes[i].rating) for i in range(batch_size)])
        distances_2 = torch.stack([(t1_nodes[i].rating-t2_nodes[i].rating) for i in range(batch_size)])
        meta_tensor = torch.from_numpy(train_data[meta].copy().astype('float32').values)
        stat_results = torch.from_numpy(train_data[stats].copy().astype('float32').values)
        opp_stat_results = torch.from_numpy(train_data[opp_stats].copy().astype('float32').values)
        target_tensor = torch.from_numpy(train_data[targets].copy().astype('float32').values)
        
        predict_X_1 = torch.cat([distances_1, meta_tensor], axis=1)
        predict_X_2 = torch.cat([distances_2, meta_tensor], axis=1)
        ## in the instance of training the update model, this update X will not be used until game n+1 (need a sequence of 2 games to train update model)
        update_X_1 = torch.cat([distances_1, meta_tensor, stat_results], axis=1)
        update_X_2 = torch.cat([distances_2, meta_tensor, opp_stat_results], axis=1)
        
        update_or_predict = np.random.random()
        
        if update_or_predict > 0.55: # slight bias to update model, it's harder
            ### PREDICT MODEL
            predictions_1 = predict_model(predict_X_1)
            predictions_2 = predict_model(predict_X_2)
            
            predictions = torch.mean(torch.stack([predictions_1, torch.Tensor([-1,-1,1])*predictions_2]), axis=0)
            predict_loss = nn.MSELoss()(predictions, target_tensor)
            predict_loss.backward()
            # Update the model's parameters
            ## for the first few iterations, let ratings stabilize
            if (j>0)&(i < 20):
                continue
            elif (j>1)&(i < 100):
                continue
            else:
                predict_optimizer.step()
            # Zero the gradients since PyTorch accumulates them
            predict_optimizer.zero_grad()
        else:
            ## UPDATE MODEL (slightly harder)
            ## use previous game's results to calculate loss of the update model
            ## get previous match information
            t1_update_X = torch.stack([t1_node.get_last_update_input() for t1_node in t1_nodes])
            t2_update_X = torch.stack([t2_node.get_last_update_input() for t2_node in t2_nodes])

            t1_update_predictions = update_model(t1_update_X)
            t2_update_predictions = update_model(t2_update_X)

            ## want both baseline predict model (no update) and predict model with updates. Compare to create loss
            baseline_X = torch.stack([t2_nodes[k].last_rating-t1_nodes[k].last_rating for k in range(len(t1_nodes))])
            update_model_X = torch.stack([t2_nodes[k].last_rating+t2_update_predictions[k] - t1_nodes[k].last_rating+t1_update_predictions[k] for k in range(len(t1_nodes))])

            ## treat this as a hypothetical prediction
            baseline_predict_X = torch.cat([baseline_X, meta_tensor], axis=1)
            update_predict_X = torch.cat([update_model_X, meta_tensor], axis=1)
            ## no result above because now being used to predict *current* match
            predict_model.eval()
            baseline_predictions = predict_model(baseline_predict_X)
            update_model_predictions = predict_model(update_predict_X)
            predict_model.train()
            baseline_MSE = nn.MSELoss()(baseline_predictions, target_tensor)
            update_model_MSE = nn.MSELoss()(update_model_predictions, target_tensor)
            ## lower is better, has to be negative if improving ratings
            update_loss = update_model_MSE-baseline_MSE
#             update_loss = nn.MSELoss()(update_model_predictions, target_tensor)
            update_loss.backward()
            ## skip the first few iterations of the later epochs to give some time for ratings to develop
            if (j>0)&(i < 25):
                continue
            elif (j>1)&(i < 100):
                continue
            else:
                update_optimizer.step()
            update_optimizer.zero_grad()
            
        ### NOW ACTUALLY UPDATE
        ## since this is n, n+1 instead of n-1, n: we can use the update model we just backprop'd
        for i in range(len(t1_nodes)):
            t1 = t1_nodes[i]
            t2 = t2_nodes[i]
            update_model.eval()
            update_pred_1 = update_model(update_X_1[i])
            update_pred_2 = update_model(update_X_2[i])
            update_model.train()
            node_update = torch.mean(torch.stack([update_pred_1, -1*update_pred_2]), axis=0)
            t1.last_rating = t1.rating.detach().clone()
            t2.last_rating = t2.rating.detach().clone()
            # Update ratings without in-place operation
            t1.rating = t1.rating.detach().clone() + node_update.detach().clone()
            t2.rating = t2.rating.detach().clone() + node_update.detach().clone()
            t1.last_vector = update_X_1[i].detach().clone()
            t2.last_vector = update_X_2[i].detach().clone()

            network_team_ratings[t1.name] = copy(t1)
            network_team_ratings[t2.name] = copy(t2)    

    ## find test error
    ## calculate val
    predict_model.eval()
    update_model.eval()
    
    network_team_ratings = {name:NetworkNode(_id, name, (torch.rand(vec_size)-0.5)*0.05) for _id, name in enumerate(teams)}
    nn_history = []
    ### have to go one game at a time
    for index, row in tqdm(data.iterrows(), total=len(data)):

        t1_id, t2_id = row['team_name'], row['opp_name']
        
        t1_nn_node = network_team_ratings[t1_id]
        t2_nn_node = network_team_ratings[t2_id]

        distance_1 = t2_nn_node.rating-t1_nn_node.rating
        distance_2 = t1_nn_node.rating-t2_nn_node.rating
        meta_tensor = torch.from_numpy(row[meta].copy().astype('float32').values)
        stat_results = torch.from_numpy(row[stats].copy().astype('float32').values)
        opp_stat_results = torch.from_numpy(row[opp_stats].copy().astype('float32').values)
        target_tensor = torch.from_numpy(row[targets].copy().astype('float32').values)

        nn_prediction_1 = predict_model(torch.cat([distance_1, meta_tensor]))
        nn_prediction_2 = predict_model(torch.cat([distance_2, meta_tensor]))
        
        nn_prediction = torch.mean(torch.stack([nn_prediction_1, torch.Tensor([-1,-1,1])*nn_prediction_2]), axis=0)
        error = target_tensor - nn_prediction
        
        p1_update_X = torch.cat([distance_1, meta_tensor, stat_results]).float()
        p2_update_X = torch.cat([distance_2, meta_tensor, opp_stat_results]).float()
        
        p1_rtg_update = update_model(p1_update_X)
        p2_rtg_update = update_model(p2_update_X)
        rtg_update = torch.mean(torch.stack([p1_rtg_update, -1*p2_rtg_update]),axis=0)
        
        id_data = [t1_nn_node._id, t1_nn_node.name, t2_nn_node._id, t2_nn_node.name]
        torch_data = torch.cat([t1_nn_node.rating.detach().clone(), t2_nn_node.rating.detach().clone(), nn_prediction.detach().clone(), target_tensor.detach().clone(), rtg_update.detach().clone()])
        id_data.extend(list(torch_data.detach().numpy()))
        nn_history.append(id_data)
        t1_nn_node.last_rating = t1_nn_node.rating.detach().clone()
        t2_nn_node.last_rating = t2_nn_node.rating.detach().clone()
        t1_nn_node.rating = t1_nn_node.rating.detach().clone() + rtg_update.detach().clone()
        t2_nn_node.rating = t2_nn_node.rating.detach().clone() + rtg_update.detach().clone()
        network_team_ratings[t1_id] = copy(t1_nn_node)
        network_team_ratings[t2_id] = copy(t2_nn_node)

    nn_hist = pd.DataFrame(nn_history, columns=['p1_id','p1_name','p2_id','p2_name',
                                                'p1_rating_x','p1_rating_y','p1_rating_z',
                                                'p2_rating_x','p2_rating_y','p2_rating_z',
                                                'prediction_score','prediction_reb','prediction_total',
                                                'result_score','result_reb','result_total',
                                                'ratings_delta_x','ratings_delta_y','ratings_delta_z'])
    
    print(nn_hist[['prediction_reb','result_reb','prediction_total','result_total','prediction_score','result_score']].corr())
    
    val_loss = mean_squared_error(nn_hist[-grade_cutoff:]['result_score'].values, nn_hist[-grade_cutoff:]['prediction_score'].values)
    print(f"NN validation MSE: {val_loss}")
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print("Saving parameters...")
        torch.save(predict_model.state_dict(), 'pairwise_predict_nn_best_model.pth')
        torch.save(update_model.state_dict(), 'pairwise_update_nn_best_model.pth')
            
            
        
        

100%|█████████████████████████████████████████████████████████████████████████| 103280/103280 [02:22<00:00, 723.03it/s]


                  prediction_reb  result_reb  prediction_total  result_total  \
prediction_reb          1.000000   -0.049878         -0.004652     -0.001618   
result_reb             -0.049878    1.000000         -0.000980      0.002453   
prediction_total       -0.004652   -0.000980          1.000000      0.047207   
result_total           -0.001618    0.002453          0.047207      1.000000   
prediction_score        0.909333   -0.058835         -0.031193     -0.002784   
result_score           -0.066438    0.527572         -0.005283      0.004536   

                  prediction_score  result_score  
prediction_reb            0.909333     -0.066438  
result_reb               -0.058835      0.527572  
prediction_total         -0.031193     -0.005283  
result_total             -0.002784      0.004536  
prediction_score          1.000000     -0.076539  
result_score             -0.076539      1.000000  
NN validation MSE: 0.016142694279551506
Saving parameters...


100%|█████████████████████████████████████████████████████████████████████████| 103280/103280 [02:23<00:00, 719.47it/s]


                  prediction_reb  result_reb  prediction_total  result_total  \
prediction_reb          1.000000   -0.005420         -0.000337     -0.004290   
result_reb             -0.005420    1.000000         -0.008254      0.002453   
prediction_total       -0.000337   -0.008254          1.000000      0.028699   
result_total           -0.004290    0.002453          0.028699      1.000000   
prediction_score        0.936148   -0.009680         -0.018825     -0.005157   
result_score           -0.006024    0.527572         -0.022657      0.004536   

                  prediction_score  result_score  
prediction_reb            0.936148     -0.006024  
result_reb               -0.009680      0.527572  
prediction_total         -0.018825     -0.022657  
result_total             -0.005157      0.004536  
prediction_score          1.000000     -0.011023  
result_score             -0.011023      1.000000  
NN validation MSE: 0.012546390295028687
Saving parameters...


100%|█████████████████████████████████████████████████████████████████████████| 103280/103280 [02:23<00:00, 717.25it/s]


...

100%|█████████████████████████████████████████████████████████████████████████| 103280/103280 [02:23<00:00, 721.37it/s]


                  prediction_reb  result_reb  prediction_total  result_total  \
prediction_reb          1.000000    0.124170         -0.010802     -0.002427   
result_reb              0.124170    1.000000         -0.007786      0.002453   
prediction_total       -0.010802   -0.007786          1.000000      0.045578   
result_total           -0.002427    0.002453          0.045578      1.000000   
prediction_score        0.997179    0.125005         -0.010509     -0.002449   
result_score            0.159730    0.527572         -0.018269      0.004536   

                  prediction_score  result_score  
prediction_reb            0.997179      0.159730  
result_reb                0.125005      0.527572  
prediction_total         -0.010509     -0.018269  
result_total             -0.002449      0.004536  
prediction_score          1.000000      0.160794  
result_score              0.160794      1.000000  
NN validation MSE: 0.011431427672505379
Saving parameters...


100%|█████████████████████████████████████████████████████████████████████████| 103280/103280 [02:27<00:00, 702.37it/s]


                  prediction_reb  result_reb  prediction_total  result_total  \
prediction_reb          1.000000    0.123144         -0.011747     -0.000918   
result_reb              0.123144    1.000000         -0.005749      0.002453   
prediction_total       -0.011747   -0.005749          1.000000      0.043345   
result_total           -0.000918    0.002453          0.043345      1.000000   
prediction_score        0.989798    0.123691         -0.010587     -0.001164   
result_score            0.153090    0.527572         -0.016254      0.004536   

                  prediction_score  result_score  
prediction_reb            0.989798      0.153090  
result_reb                0.123691      0.527572  
prediction_total         -0.010587     -0.016254  
result_total             -0.001164      0.004536  
prediction_score          1.000000      0.154446  
result_score              0.154446      1.000000  
NN validation MSE: 0.011510206386446953


 19%|█████████████▊                                                            | 19263/103280 [00:27<02:00, 699.49it/s]

KeyboardInterrupt



In [ ]:
import seaborn as sns

sns.displot(nn_hist.prediction_score)


In [ ]:
nn_hist[['prediction_score','result_score']].corr()


In [ ]:
nn_hist[['prediction_reb','result_reb','prediction_total','result_total','prediction_score','result_score']].corr()

In [ ]:
sns.histplot(nn_hist.result_total)

In [ ]:
sns.histplot(nn_hist.prediction_total)


In [ ]:

nn_hist

